In [1]:
# Import necessary libraries
import numpy as np
import math
import carla
import time
from collections import deque


In [2]:
# Constants for the EKF
dt = 0.1  # Time step (can be adjusted based on your simulation time)
R_imu = np.eye(3) * 0.02  # IMU noise covariance (tuned)
R_lidar = np.eye(2) * 0.05  # Lidar noise covariance (tuned)

# State Vector: [x, y, theta, v, ax, alpha] (position, velocity, orientation, acceleration, angular velocity)
state = np.zeros(6)
P = np.eye(6)  # Covariance matrix



In [3]:
# Define GPS to Cartesian conversion function
def gps_to_cartesian(lat, lon, lat_ref, lon_ref):
    # Convert geographical coordinates (latitude, longitude) to CARLA coordinates (x, y)
    R = 6371000  # Radius of the Earth in meters
    dx = (lon - lon_ref) * R * np.cos(lat_ref * np.pi / 180)
    dy = (lat - lat_ref) * R
    return np.array([dx, dy])

# Global variables for GPS reference
lat_ref, lon_ref = None, None
reference_set = False  # Flag to ensure the reference is set only once

In [4]:
# Setup CARLA client and world
client = carla.Client('localhost', 2000)
world = client.get_world()

# Get the vehicle blueprint
world = client.get_world()
blueprint_library = world.get_blueprint_library()
vehicle_bp = blueprint_library.find('vehicle.tesla.model3')
spawn_point = world.get_map().get_spawn_points()[0]
vehicle = world.spawn_actor(vehicle_bp, spawn_point)
# Enable autopilot on the vehicle
vehicle.set_autopilot(True)  # This enables CARLA's autopilot control

# Create IMU and LiDAR sensors
imu_bp = world.get_blueprint_library().find('sensor.other.imu')
imu_transform = carla.Transform(carla.Location(x=1.5, z=2.5))  # Adjust the sensor position
imu_sensor = world.spawn_actor(imu_bp, imu_transform, attach_to=vehicle)

lidar_bp = world.get_blueprint_library().find('sensor.lidar.ray_cast')
lidar_transform = carla.Transform(carla.Location(x=1.5, z=2.5))  # Adjust the sensor position
lidar_sensor = world.spawn_actor(lidar_bp, lidar_transform, attach_to=vehicle)


In [5]:

# GPS callback function (only for error calculation, not state update)
def gps_callback(data):
    global lat_ref, lon_ref, reference_set, state

    # Extract GPS data
    lat, lon = data.latitude, data.longitude

    # Set the reference point on the first callback
    if not reference_set:
        lat_ref, lon_ref = lat, lon
        # Align EKF's initial position to the reference GPS
        state[:2] = gps_to_cartesian(lat, lon, lat_ref, lon_ref)
        reference_set = True
        print(f"Reference set to GPS (lat: {lat_ref}, lon: {lon_ref})")

    # Convert current GPS data to Cartesian coordinates
    gps_position = gps_to_cartesian(lat, lon, lat_ref, lon_ref)

    # Compute and print the error
    error = np.linalg.norm(state[:2] - gps_position)
    print(f"Position error between GPS and EKF: {error:.2f} meters")

In [6]:
# IMU callback function
def imu_callback(data):
    global state, P
    ax = data.accelerometer.x
    ay = data.accelerometer.y
    az = data.accelerometer.z
    alpha = data.gyroscope.z  # Angular velocity (z-axis)
    
    # IMU update step (prediction based on motion model)
    # State prediction (predict next state based on IMU)
    state[3] = ax * dt  # Update velocity based on acceleration
    state[4] = ax  # Update acceleration
    state[5] = alpha  # Update angular velocity
    
    # EKF Prediction
    F = np.eye(6)  # Linearization of the motion model (simplified here)
    Q = np.eye(6) * 0.1  # Process noise
    P = F @ P @ F.T + Q  # Prediction of covariance matrix
    state = state + np.array([state[3]*dt, state[4]*dt, state[5]*dt, 0, 0, 0])  # Simple prediction update


In [7]:
# LiDAR callback function
def lidar_callback(data):
    global state, P
    points = np.frombuffer(data.raw_data, dtype=np.float32).reshape(-1, 4)
    
    # Estimate distance from number of points
    distance = len(points) * 0.01  # Assuming a constant scaling factor for simplicity
    
    # LiDAR measurement model (mapping to state)
    z = np.array([state[0] + distance * np.cos(state[2]),  # x update
                  state[1] + distance * np.sin(state[2])])  # y update
    
    # Measurement matrix (for LiDAR update)
    H = np.array([[1, 0, 0, 0, 0, 0],
                  [0, 1, 0, 0, 0, 0]])
    
    # Measurement residual
    y = z - H @ state  # Difference between predicted and actual measurement
    S = H @ P @ H.T + R_lidar  # Measurement covariance
    K = P @ H.T @ np.linalg.inv(S)  # Kalman gain
    
    # Update only the position (x, y) in the state vector
    state[:2] = state[:2] + K[:2, :] @ y  # Update position (x, y)
    
    # Update the covariance matrix
    P = (np.eye(6) - K @ H) @ P


In [8]:
# # Constants for GPS to Local transformation
# R = 6371000  # Earth's radius in meters

# # Get the vehicle's initial location in CARLA
# initial_location = vehicle.get_location()  
# carla_x_ref = initial_location.x  # Reference CARLA x
# carla_y_ref = initial_location.y  # Reference CARLA y

# # Use the initial GPS location as the reference
# gps_data = gps_sensor.get_data()  # Assuming you can access an initial GPS reading
# lat_ref, lon_ref = gps_data.latitude, gps_data.longitude  # Reference for GPS to Cartesian conversion

# # Functions for coordinate transformation
# def gps_to_carla(lat, lon, lat_ref, lon_ref, carla_x_ref, carla_y_ref):
#     """
#     Converts GPS coordinates (lat, lon) to CARLA coordinates (x, y).
#     """
#     dx = (lon - lon_ref) * R * np.cos(lat_ref * np.pi / 180)
#     dy = (lat - lat_ref) * R
#     carla_x = carla_x_ref + dx
#     carla_y = carla_y_ref + dy
#     return np.array([carla_x, carla_y])

In [9]:
# # GPS callback function (for error calculation only)
# def gps_callback(data):
#     global state
#     lat = data.latitude
#     lon = data.longitude

#     # Convert GPS data to CARLA coordinates (x, y) using the reference point
#     gps_position = gps_to_carla(lat, lon, lat_ref, lon_ref, carla_x_ref, carla_y_ref)

#     # Calculate the error between EKF position and GPS position
#     error = np.linalg.norm(state[:2] - gps_position)

#     # Print the error
#     print(f"Position error between GPS and EKF: {error:.2f} meters")


In [10]:
# Attach sensors to the vehicle
imu_sensor.listen(lambda data: imu_callback(data))
lidar_sensor.listen(lambda data: lidar_callback(data))

# Create the GPS sensor and listen for data
gps_bp = world.get_blueprint_library().find('sensor.other.gnss')
gps_transform = carla.Transform(carla.Location(x=1.5, z=2.5))  # Adjust sensor position
gps_sensor = world.spawn_actor(gps_bp, gps_transform, attach_to=vehicle)
gps_sensor.listen(lambda data: gps_callback(data))

In [11]:
# def calculate_accuracy(ekf_position, gps_position):
#     # Calculate the error (distance) between EKF and GPS positions in 2D (x, y)
#     error = np.linalg.norm(ekf_position[:2] - gps_position)  # Only use x and y for error calculation
#     print(f"Accuracy/Error (meters): {error:.2f}")


Reference set to GPS (lat: 0.0006966968512500671, lon: -5.8378178402185456e-05)
Position error between GPS and EKF: 0.00 meters


In [12]:
# Simulate for 100 seconds
for _ in range(100):
    world.tick()  # Advance the simulation by one tick
    time.sleep(dt)
    
    # Print out the current state and covariance matrix
    print("Current state:", state)
    print("Covariance Matrix P:", P)

Position error between GPS and EKF: 2.68 meters
Position error between GPS and EKF: 4.96 meters
Position error between GPS and EKF: 6.68 meters
Position error between GPS and EKF: 8.18 meters
Position error between GPS and EKF: 8.18 meters
Position error between GPS and EKF: 9.66 meters
Position error between GPS and EKF: 12.99 meters
Position error between GPS and EKF: 14.53 meters
Current state: [ 1.45301123e+01 -1.39499068e-02 -5.29865202e-10  0.00000000e+00
  0.00000000e+00 -1.18374177e-09]
Covariance Matrix P: [[0.13660254 0.         0.         0.         0.         0.        ]
 [0.         0.13660254 0.         0.         0.         0.        ]
 [0.         0.         2.1        0.         0.         0.        ]
 [0.         0.         0.         2.1        0.         0.        ]
 [0.         0.         0.         0.         2.1        0.        ]
 [0.         0.         0.         0.         0.         2.1       ]]
Position error between GPS and EKF: 16.16 meters
Position error 

Position error between GPS and EKF: 131.60 meters
Position error between GPS and EKF: 133.05 meters
Current state: [142.14690506   6.39317079  -0.38411148   0.36206794   3.62067938
  -0.30659437]
Covariance Matrix P: [[0.13660254 0.         0.         0.         0.         0.        ]
 [0.         0.13660254 0.         0.         0.         0.        ]
 [0.         0.         8.2        0.         0.         0.        ]
 [0.         0.         0.         8.2        0.         0.        ]
 [0.         0.         0.         0.         8.2        0.        ]
 [0.         0.         0.         0.         0.         8.2       ]]
Position error between GPS and EKF: 134.47 meters
Position error between GPS and EKF: 135.84 meters
Position error between GPS and EKF: 137.24 meters
Position error between GPS and EKF: 137.38 meters
Position error between GPS and EKF: 140.52 meters
Position error between GPS and EKF: 142.10 meters
Current state: [153.32192698   3.07233966  -0.51970926   0.27338667 

Position error between GPS and EKF: 311.18 meters
Position error between GPS and EKF: 316.70 meters
Position error between GPS and EKF: 321.94 meters
Position error between GPS and EKF: 327.42 meters
Position error between GPS and EKF: 332.77 meters
Current state: [ 2.41511715e+02 -6.34906638e+01 -6.46517861e-01  1.30100906e-02
  1.30100906e-01  1.03329621e-01]
Covariance Matrix P: [[ 0.13660254  0.          0.          0.          0.          0.        ]
 [ 0.          0.13660254  0.          0.          0.          0.        ]
 [ 0.          0.         14.2         0.          0.          0.        ]
 [ 0.          0.          0.         14.2         0.          0.        ]
 [ 0.          0.          0.          0.         14.2         0.        ]
 [ 0.          0.          0.          0.          0.         14.2       ]]
Position error between GPS and EKF: 337.98 meters
Position error between GPS and EKF: 345.07 meters
Position error between GPS and EKF: 353.56 meters
Position error

Position error between GPS and EKF: 641.32 meters
Position error between GPS and EKF: 647.31 meters
Position error between GPS and EKF: 653.22 meters
Current state: [ 3.47148074e+02 -1.08000137e+02 -1.84263111e-01 -6.05353117e-03
 -6.05353117e-02  4.53104898e-02]
Covariance Matrix P: [[ 0.13660254  0.          0.          0.          0.          0.        ]
 [ 0.          0.13660254  0.          0.          0.          0.        ]
 [ 0.          0.         20.1         0.          0.          0.        ]
 [ 0.          0.          0.         20.1         0.          0.        ]
 [ 0.          0.          0.          0.         20.1         0.        ]
 [ 0.          0.          0.          0.          0.         20.1       ]]
Position error between GPS and EKF: 658.50 meters
Position error between GPS and EKF: 663.73 meters
Position error between GPS and EKF: 668.62 meters
Position error between GPS and EKF: 673.87 meters
Position error between GPS and EKF: 679.65 meters
Position error

Position error between GPS and EKF: 1023.20 meters
Position error between GPS and EKF: 1028.73 meters
Position error between GPS and EKF: 1034.51 meters
Position error between GPS and EKF: 1042.18 meters
Position error between GPS and EKF: 1047.89 meters
Current state: [ 4.75413088e+02 -1.19192437e+02 -2.66964939e-02 -3.73142213e-03
 -3.73142213e-02  1.06585296e-02]
Covariance Matrix P: [[ 0.13660254  0.          0.          0.          0.          0.        ]
 [ 0.          0.13660254  0.          0.          0.          0.        ]
 [ 0.          0.         26.7         0.          0.          0.        ]
 [ 0.          0.          0.         26.7         0.          0.        ]
 [ 0.          0.          0.          0.         26.7         0.        ]
 [ 0.          0.          0.          0.          0.         26.7       ]]
Position error between GPS and EKF: 1054.79 meters
Position error between GPS and EKF: 1061.88 meters
Position error between GPS and EKF: 1068.58 meters
Positi

...

Position error between GPS and EKF: 7501.22 meters
Position error between GPS and EKF: 7501.23 meters
Position error between GPS and EKF: 7502.70 meters
Position error between GPS and EKF: 7505.92 meters
Position error between GPS and EKF: 7505.92 meters
Position error between GPS and EKF: 7509.20 meters
Position error between GPS and EKF: 7510.62 meters
Position error between GPS and EKF: 7510.63 meters
Position error between GPS and EKF: 7513.56 meters
Position error between GPS and EKF: 7513.56 meters
Position error between GPS and EKF: 7516.53 meters
Position error between GPS and EKF: 7518.09 meters
Position error between GPS and EKF: 7519.60 meters
Position error between GPS and EKF: 7521.22 meters
Position error between GPS and EKF: 7521.22 meters
Position error between GPS and EKF: 7523.33 meters
Position error between GPS and EKF: 7526.68 meters
Position error between GPS and EKF: 7528.14 meters
Position error between GPS and EKF: 7528.14 meters
Position error between GPS and 

Position error between GPS and EKF: 7752.54 meters
Position error between GPS and EKF: 7752.55 meters
Position error between GPS and EKF: 7754.23 meters
Position error between GPS and EKF: 7757.18 meters
Position error between GPS and EKF: 7757.18 meters
Position error between GPS and EKF: 7760.48 meters
Position error between GPS and EKF: 7762.06 meters
Position error between GPS and EKF: 7763.50 meters
Position error between GPS and EKF: 7765.07 meters
Position error between GPS and EKF: 7766.56 meters
Position error between GPS and EKF: 7767.99 meters
Position error between GPS and EKF: 7767.99 meters
Position error between GPS and EKF: 7771.36 meters
Position error between GPS and EKF: 7771.35 meters
Position error between GPS and EKF: 7774.04 meters
Position error between GPS and EKF: 7775.74 meters
Position error between GPS and EKF: 7777.35 meters
Position error between GPS and EKF: 7778.83 meters
Position error between GPS and EKF: 7778.82 meters
Position error between GPS and 

Position error between GPS and EKF: 8020.33 meters
Position error between GPS and EKF: 8020.33 meters
Position error between GPS and EKF: 8023.59 meters
Position error between GPS and EKF: 8023.59 meters
Position error between GPS and EKF: 8026.53 meters
Position error between GPS and EKF: 8026.53 meters
Position error between GPS and EKF: 8029.83 meters
Position error between GPS and EKF: 8031.42 meters
Position error between GPS and EKF: 8033.06 meters
Position error between GPS and EKF: 8034.44 meters
Position error between GPS and EKF: 8034.43 meters
Position error between GPS and EKF: 8037.55 meters
Position error between GPS and EKF: 8037.55 meters
Position error between GPS and EKF: 8039.28 meters
Position error between GPS and EKF: 8041.76 meters
Position error between GPS and EKF: 8043.16 meters
Position error between GPS and EKF: 8044.66 meters
Position error between GPS and EKF: 8046.45 meters
Position error between GPS and EKF: 8047.98 meters
Position error between GPS and 

Position error between GPS and EKF: 8409.33 meters
Position error between GPS and EKF: 8415.41 meters
Position error between GPS and EKF: 8420.04 meters
Position error between GPS and EKF: 8424.22 meters
Position error between GPS and EKF: 8428.63 meters
Position error between GPS and EKF: 8433.32 meters
Position error between GPS and EKF: 8437.93 meters
Position error between GPS and EKF: 8442.49 meters
Position error between GPS and EKF: 8446.98 meters
Position error between GPS and EKF: 8449.87 meters
Position error between GPS and EKF: 8454.13 meters
Position error between GPS and EKF: 8460.06 meters
Position error between GPS and EKF: 8464.52 meters
Position error between GPS and EKF: 8467.49 meters
Position error between GPS and EKF: 8471.88 meters
Position error between GPS and EKF: 8477.85 meters
Position error between GPS and EKF: 8482.67 meters
Position error between GPS and EKF: 8487.27 meters
Position error between GPS and EKF: 8491.81 meters
Position error between GPS and 

Position error between GPS and EKF: 9247.93 meters
Position error between GPS and EKF: 9254.80 meters
Position error between GPS and EKF: 9262.76 meters
Position error between GPS and EKF: 9269.71 meters
Position error between GPS and EKF: 9277.13 meters
Position error between GPS and EKF: 9282.77 meters
Position error between GPS and EKF: 9288.64 meters
Position error between GPS and EKF: 9295.11 meters
Position error between GPS and EKF: 9302.17 meters
Position error between GPS and EKF: 9308.22 meters
Position error between GPS and EKF: 9313.79 meters
Position error between GPS and EKF: 9319.40 meters
Position error between GPS and EKF: 9324.78 meters
Position error between GPS and EKF: 9330.06 meters
Position error between GPS and EKF: 9335.45 meters
Position error between GPS and EKF: 9340.80 meters
Position error between GPS and EKF: 9344.45 meters
Position error between GPS and EKF: 9350.05 meters
Position error between GPS and EKF: 9357.75 meters
Position error between GPS and 

In [ ]:
imu_sensor.stop()
lidar_sensor.stop()
vehicle.destroy()